In [1]:
import datetime


import spacy
from spacy import displacy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#to speed up pandas operands
from pandarallel import pandarallel

In [2]:
reviews_df = pd.read_csv("../Data/Reviews.csv")

In [3]:
reviews_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
reviews_df.shape

(568454, 10)

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
specific_text = reviews_df.iloc[0,9]

In [7]:
specific_text

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [8]:
specific_doc = nlp(specific_text)

In [9]:
print([token.text for token in specific_doc])

['I', 'have', 'bought', 'several', 'of', 'the', 'Vitality', 'canned', 'dog', 'food', 'products', 'and', 'have', 'found', 'them', 'all', 'to', 'be', 'of', 'good', 'quality', '.', 'The', 'product', 'looks', 'more', 'like', 'a', 'stew', 'than', 'a', 'processed', 'meat', 'and', 'it', 'smells', 'better', '.', 'My', 'Labrador', 'is', 'finicky', 'and', 'she', 'appreciates', 'this', 'product', 'better', 'than', ' ', 'most', '.']


In [10]:
for token in specific_doc:
    print("{:8}\t{:8}\t{:7}\t{:5}\t{}".format(token.text, token.lemma_,token.pos_,token.tag_,token.dep_))

I       	-PRON-  	PRON   	PRP  	nsubj
have    	have    	AUX    	VBP  	aux
bought  	buy     	VERB   	VBN  	ROOT
several 	several 	ADJ    	JJ   	dobj
of      	of      	ADP    	IN   	prep
the     	the     	DET    	DT   	det
Vitality	Vitality	PROPN  	NNP  	compound
canned  	can     	VERB   	VBN  	compound
dog     	dog     	NOUN   	NN   	compound
food    	food    	NOUN   	NN   	compound
products	product 	NOUN   	NNS  	pobj
and     	and     	CCONJ  	CC   	cc
have    	have    	AUX    	VBP  	aux
found   	find    	VERB   	VBN  	conj
them    	-PRON-  	PRON   	PRP  	nsubj
all     	all     	DET    	DT   	appos
to      	to      	PART   	TO   	aux
be      	be      	AUX    	VB   	ccomp
of      	of      	ADP    	IN   	prep
good    	good    	ADJ    	JJ   	amod
quality 	quality 	NOUN   	NN   	pobj
.       	.       	PUNCT  	.    	punct
The     	the     	DET    	DT   	det
product 	product 	NOUN   	NN   	nsubj
looks   	look    	VERB   	VBZ  	ROOT
more    	more    	ADV    	RBR  	advmod
like    	like    	SCO

In [11]:
print("Verbs present tense:")
present_tense_verbs = [token.text for token in specific_doc if token.tag_ == "VB" or token.tag_ == "VBG" or token.tag_ == "VBD"] 
print(present_tense_verbs)

Verbs present tense:
['be']


In [12]:
print([token.text for token in specific_doc if token.dep_=="ROOT" or token.dep_=="pobj"])

['bought', 'products', 'quality', 'looks', 'stew', 'meat', 'is']


Keywords I want to keep track of:
* buy
* return
* like
* love
* hate
* dislike
* best
* worst


In [ ]:
reviews_df['spacy_doc'] = list(nlp.pipe(reviews_df.Text))

In [ ]:
#create spacy docs for each review store in the dataframe

pandarallel.initialize()

start_time = datetime.datetime.now()
def create_nlp_doc(df_row):
    return nlp(df_row["Text"])

reviews_df["spacy_doc"] = reviews_df.parallel_apply(create_nlp_doc,axis=1)

end_time = datetime.datetime.now()

#start_time = datetime.datetime.now()

#reviews_df["spacy_doc"] =  list(nlp.pipe(reviews_df.Text))

#end_time = datetime.datetime.now()
print("time spent doing operation is {}".format(end_time - start_time))

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
#TODO: parallelize the cell above